In [14]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

# This code Scraps a single query from Amazon | Purpose modifed

from bs4 import BeautifulSoup as soup
import requests
import os
import csv

# vars with GLOBAL scope
bot_query = "https://www.amazon.in/s?k="  # Defualt order, no filters are being applied
# HEADER = {
#     "User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"
# }
HEADER={"User-Agent": "Python-urlli~"}
PROXY = {
    "https": "138.0.207.18:58566"
}  # {"https": "https//124.107.229.210:8080", "http":"http//124.107.229.210:8080"} | Proxy Rotation is absent | pip package will be released sooner
csv_index = 0  # To keep propering indexing when adding data from different pages

## Error handling needed when image sizes mismatch
def csv_append(
    user_query="-",
    item_count="-",
    name="-",
    item_page="-",
    rating="-",
    review="-",
    is1="-",
    is15="-",
    is2="-",
    is25="-",
    is3="-",
    price="-",
):
    global is_first
    global csv_index

    csv_index += 1
    item_count = csv_index  # Overtake
    csv_name = user_query.replace("+", "-") + ".csv"
    if is_first:
        with open(csv_name, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    "SN",
                    "Name",
                    "Link to item",
                    "Rating",
                    "Reviews",
                    "Image Download link 1x",
                    "Image Download link 1.5x",
                    "Image Download link 2x",
                    "Image Download link 2.5x",
                    "Image Download link 3x",
                    "Price",
                ]
            )
            writer.writerow(
                [
                    item_count,
                    name,
                    item_page,
                    rating,
                    review.replace(",", ""),
                    is1,
                    is15,
                    is2,
                    is25,
                    is3,
                    price,
                ]
            )
    else:
        with open(csv_name, "a+", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    item_count,
                    name,
                    item_page,
                    rating,
                    review.replace(",", ""),
                    is1,
                    is15,
                    is2,
                    is25,
                    is3,
                    price,
                ]
            )


def slugify(img_name):
    """
    Slug Generator with image format
    :param1: image-name
    :return: slugged and format typed image
    """

    sluged_name = (
        img_name.replace(" ", "-").replace('"', "").replace("(", " ").replace(")", " ")
    )
    sluged_name = (
        sluged_name.replace("|", " ").replace("&", " ").replace(":", " ")
    )  # ASCII Range can be checked instead, More info needed about how will the images \
    # be used.

    return "{}.{}".format(sluged_name, "jpg")


def img_download(url, img_name):  # img_name):
    """
    Downloads the image from given url
    :param1: url
    :param2: imgage-name
    """
    # yes change here
    global csv_index

    # img_name = slugify(img_name).encode('ascii', 'ignore')
    img_name = slugify(str(csv_index))
    img = requests.get(url, headers=HEADER)
    with open(img_name, "wb") as f:  # Put them in folder properly
        f.write(img.content)


def scrape_bundle(user_query, page_count):

    global is_first
    is_first = True
    for count in range(1, int(page_count) + 1):
        scrape_query(user_query, count)
        print(count)
        is_first = False


def scrape_query(user_query, page_count):
    """
    Scrape and store the details of first page in csv that is when we search an item on amazon. Image is also downloaded.
    :param1: Name of item
    :return: None if no item appears
    Scraped Data:: Link to item | Item Name | Review count | Item Rating | Image download links | Item Availability | Price
    """

    global is_first

    amazon_query = bot_query + user_query + "&" + "page=" + str(page_count)

    client = requests.get(amazon_query, headers=HEADER)  # proxies=PROXY | gimmeproxy
    page_html = client.text

    page_soup = soup(page_html, "html.parser")
    container = page_soup.findAll("div", {"class": "s-result-item"})

    bundled_data = []

    ## Function can be made for below code | too much arg passing
    ## Better Error Handling is needed | can be avoided by key check
    # Taking only first item | Run complete loop to get all single page results
    if len(container) > 0:
        item_count = 0
        for item in container:
            try:

                item_count += 1
                item_page = "https://www.amazon.in" + item.a["href"]
                print(item_page)

                try:
                    image_srcset = [
                        (img_link.split(" ")[0], img_link.split(" ")[1])
                        for img_link in item.a.img["srcset"].split(", ")
                    ]
                except:
                    image_srcset = None
                try:
                    img_alt = item.img["alt"]  # item-name
                except:
                    img_alt = ""
                try:
                    rating = item.span.find("span", {"class": "a-icon-alt"}).text
                    review_count = item.span.find("span", {"class": "a-size-base"}).text
                except:
                    rating = "-"
                    review_count = "-"

                # Availability
                for x in item.span.find_all("span"):
                    if x.text == "Currently unavailable.":
                        availability = "Currently unavailable."
                else:
                    availability = "Available"

                price = item.span.find("span", {"class": "a-price-whole"})
                if price != None:
                    price = price.text
                else:
                    price = "-"

                # Downloading most HR image
                try:
                    img_download(url=image_srcset[-1][0], img_name=img_alt)
                except Exception as e:
                    print(str(e))

                try:
                    is1 = image_srcset[0][0]
                    is15 = image_srcset[1][0]
                    is2 = image_srcset[2][0]
                    is25 = image_srcset[3][0]
                except:
                    is1 = "-"
                    is15 = "-"
                    is2 = "-"
                    is25 = "-"
                try:  # Maybe 1 Image we have
                    is3 = image_srcset[-1][0]
                except:
                    is3 = "-"

                csv_append(
                    user_query=user_query,
                    item_count=item_count,
                    name=img_alt,
                    item_page=item_page,
                    rating=rating,
                    review=review_count,
                    is1=is1,
                    is15=is15,
                    is2=is2,
                    is25=is25,
                    is3=is3,
                    price=price,
                )
                is_first = False
            except:
                pass
    else:
        item = None
        return item


if __name__ == "__main__":

    user_query = input("Enter your Product: ").replace(
        " ", "+"
    )  # More filtering is needed | search 'mobile' to get all mobile data scraped
    page_count = input("Scrape upto how many pages: ")

    if page_count in ["0", " ", None]:
        scrape_bundle(user_query, "1")
    else:
        scrape_bundle(user_query, page_count)

Enter your Product: men tshirt
Scrape upto how many pages: 100
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09057221LXP5YSG6H562&url=%2FAdidas-Solid-Regular-T-Shirt-FP7892_S_Scarlet%2Fdp%2FB0829L2SX3%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915078%26sr%3D8-1-spons%26psc%3D1&qualifier=1596915077&id=6018560220667272&widgetName=sp_atf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09076303JPACAEB5IK4Y&url=%2FAdidas-Printed-Regular-T-Shirt-FM4791_M_Black%2Fdp%2FB081TTNJMQ%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915078%26sr%3D8-2-spons%26psc%3D1&qualifier=1596915077&id=6018560220667272&widgetName=sp_atf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09272782H88EULH86LJK&url=%2FAdidas-Solid-Regular-T-Shirt-FJ5148_2XL_Black%2Fdp%2FB081TP37KW%2Fref%3Dsr_1_3_sspa%3Fdchild%3D1%26keyw

https://www.amazon.in/Scott-International-Cotton-T-shirts-XX-Large/dp/B06X3V3P7C/ref=sr_1_45?dchild=1&keywords=men+tshirt&qid=1596915078&sr=8-45
https://www.amazon.in/BULLMER-Cotton-Printed-T-shirt-Multicolour/dp/B0892VNQP7/ref=sr_1_46?dchild=1&keywords=men+tshirt&qid=1596915078&sr=8-46
https://www.amazon.in/Adidas-Solid-Regular-T-Shirt-FI7941_White_L/dp/B07VFP7M4X/ref=sr_1_47?dchild=1&keywords=men+tshirt&qid=1596915078&sr=8-47
https://www.amazon.in/London-Hills-Solid-Sleeve-T-Shirts/dp/B07FLT438N/ref=sr_1_48?dchild=1&keywords=men+tshirt&qid=1596915078&sr=8-48
https://www.amazon.in/Wrangler-Solid-T-Shirt-W3114523335Z_JSW-Navy_XL/dp/B07MQ5QQKC/ref=sr_1_49?dchild=1&keywords=men+tshirt&qid=1596915078&sr=8-49
https://www.amazon.in/Association-%E0%A4%AA%E0%A5%81%E0%A4%B0%E0%A5%81%E0%A4%B7%E0%A5%8B%E0%A4%82-%E0%A4%B8%E0%A5%89%E0%A4%B2%E0%A4%BF%E0%A4%A1-%E0%A4%B0%E0%A5%87%E0%A4%97%E0%A5%81%E0%A4%B2%E0%A4%B0-%E0%A4%9F%E0%A5%80-%E0%A4%B6%E0%A4%B0%E0%A5%8D%E0%A4%9F/dp/B0793HPJTD/ref=sr_1_50?dchi

https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_aps_sr_pg2_1?ie=UTF8&adId=A0906614KCD5A570CK40&url=%2FAdidas-Plain-Regular-T-Shirt-FM6027_Black%2Fdp%2FB0823FFP23%2Fref%3Dsr_1_71_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915101%26sr%3D8-71-spons%26psc%3D1&qualifier=1596915101&id=2396617890414488&widgetName=sp_mtf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_aps_sr_pg2_1?ie=UTF8&adId=A09057221LXP5YSG6H562&url=%2FAdidas-Solid-Regular-T-Shirt-FP7892_S_Scarlet%2Fdp%2FB0829L2SX3%2Fref%3Dsr_1_72_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915101%26sr%3D8-72-spons%26psc%3D1&qualifier=1596915101&id=2396617890414488&widgetName=sp_mtf
https://www.amazon.in/Wexford-Cotton-T-Shirt-WEX-WFE010A-X-Large-X-Large/dp/B01CTVC2CG/ref=sr_1_73?dchild=1&keywords=men+tshirt&qid=1596915101&sr=8-73
https://www.amazon.in/EYEBOGLER-Regular-T-Shirt-L-EBT317-TMBL-Melange-Black/dp/B08121HYCH/ref=sr_1_74?dchild=1&keywords=men+tshirt&qid=1596915

https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg3_1?ie=UTF8&adId=A0729216ZSVUU2WKHUDX&url=%2FAD-AV-641_Mens_Tshirt_BLACKSTRIPE_A%2Fdp%2FB07D7B488B%2Fref%3Dsr_1_100_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915123%26sr%3D8-100-spons%26psc%3D1&qualifier=1596915123&id=742982752744424&widgetName=sp_atf_next
https://www.amazon.in/Katso-Cotton-Hooded-T-Shirt-Black-White/dp/B0867GSXVP/ref=sr_1_101?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-101
https://www.amazon.in/Chromozome-OS8-Mens-T-Shirt-8902733373777_OS8-White-L/dp/B073ZKDH4W/ref=sr_1_102?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-102
https://www.amazon.in/US-Polo-Association-Striped-I688-018-ST/dp/B0819ZYKSL/ref=sr_1_103?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-103
https://www.amazon.in/Zesteez-Yoga-T-Shirt-Black-Medium/dp/B079RGY6VM/ref=sr_1_104?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-104
https://www.amazon.in/blackberrys-Printed-T-Shirt-ETCC0951R3MS19FL42

https://www.amazon.in/pluss-Striped-Regular-T-Shirt-MTSHS5707-NEUTRALGREY_Multi_3XL/dp/B07VFHRXBW/ref=sr_1_148?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-148
https://www.amazon.in/EASY-BUY-T-Shirt-WT91GA38MT_-Melange/dp/B08C89HR5P/ref=sr_1_149?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-149
https://www.amazon.in/AWG-Weather-Regular-T-Shirt-AWGDFT-BL-SBU-DGR-L_Multicolour_Large/dp/B07PCQ7H28/ref=sr_1_150?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-150
https://www.amazon.in/US-Polo-Association-T-Shirt-USTS6162_Navy_L/dp/B07VP41H5N/ref=sr_1_151?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-151
https://www.amazon.in/Jockey-Plain-Regular-T-Shirt-2726_Charcoal_XX-Large/dp/B072MLQBCT/ref=sr_1_152?dchild=1&keywords=men+tshirt&qid=1596915123&sr=8-152
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg3_1?ie=UTF8&adId=A06690081G0CMBD6ZVUGG&url=%2FElements-Peter-England-Plain-PAKPCSGBF69907_White%2Fdp%2FB0825BNLWL%2Fref%3Dsr_1_153_sspa%3Fdchild%3D1%

...

https://www.amazon.in/gp/help/customer/display.html?nodeId=200533820
'NoneType' object is not subscriptable
92
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg93_1?ie=UTF8&adId=A00684273SZPO31QF6IVU&url=%2FLee-X-Line-Printed-T-Shirt-L41345CB0XLN_Jsw-Akr%2Fdp%2FB07WKDR86K%2Fref%3Dsr_1_4417_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915682%26sr%3D8-4417-spons%26psc%3D1&qualifier=1596915682&id=4963271519731850&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg93_1?ie=UTF8&adId=A05556532YMG0ZIDRQ93S&url=%2FVeirdo-Sleeve-Round-Cotton-T-Shirt%2Fdp%2FB07VPW43VK%2Fref%3Dsr_1_4418_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915682%26sr%3D8-4418-spons%26psc%3D1&qualifier=1596915682&id=4963271519731850&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg93_1?ie=UTF8&adId=A059929715OJ26ZFDI0F9&url=%2FAmazon-Brand-Regular-T-S

https://www.amazon.in/gp/help/customer/display.html?nodeId=200533820
'NoneType' object is not subscriptable
94
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg95_1?ie=UTF8&adId=A00684273SZPO31QF6IVU&url=%2FLee-X-Line-Printed-T-Shirt-L41345CB0XLN_Jsw-Akr%2Fdp%2FB07WKDR86K%2Fref%3Dsr_1_4513_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915697%26sr%3D8-4513-spons%26psc%3D1&qualifier=1596915697&id=2769406411656883&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg95_1?ie=UTF8&adId=A05556532YMG0ZIDRQ93S&url=%2FVeirdo-Sleeve-Round-Cotton-T-Shirt%2Fdp%2FB07VPW43VK%2Fref%3Dsr_1_4514_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915697%26sr%3D8-4514-spons%26psc%3D1&qualifier=1596915697&id=2769406411656883&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg95_1?ie=UTF8&adId=A059929715OJ26ZFDI0F9&url=%2FAmazon-Brand-Regular-T-S

https://www.amazon.in/gp/help/customer/display.html?nodeId=200533820
'NoneType' object is not subscriptable
96
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg97_1?ie=UTF8&adId=A00684273SZPO31QF6IVU&url=%2FLee-X-Line-Printed-T-Shirt-L41345CB0XLN_Jsw-Akr%2Fdp%2FB07WKDR86K%2Fref%3Dsr_1_4609_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915705%26sr%3D8-4609-spons%26psc%3D1&qualifier=1596915704&id=4974290502177358&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg97_1?ie=UTF8&adId=A05556532YMG0ZIDRQ93S&url=%2FVeirdo-Sleeve-Round-Cotton-T-Shirt%2Fdp%2FB07VPW43VK%2Fref%3Dsr_1_4610_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915705%26sr%3D8-4610-spons%26psc%3D1&qualifier=1596915704&id=4974290502177358&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg97_1?ie=UTF8&adId=A059929715OJ26ZFDI0F9&url=%2FAmazon-Brand-Regular-T-S

https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg98_1?ie=UTF8&adId=A02876552IHU0KNRLSWRY&url=%2FLee-X-Line-Printed-T-Shirt-L41368CB0XLN_Jsw-Navy%2Fdp%2FB07WG5V1JV%2Fref%3Dsr_1_4668_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915710%26sr%3D8-4668-spons%26psc%3D1&qualifier=1596915710&id=7018803508843260&widgetName=sp_btf
https://www.amazon.in/gp/help/customer/display.html?nodeId=200533820
'NoneType' object is not subscriptable
98
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg99_1?ie=UTF8&adId=A00684273SZPO31QF6IVU&url=%2FLee-X-Line-Printed-T-Shirt-L41345CB0XLN_Jsw-Akr%2Fdp%2FB07WKDR86K%2Fref%3Dsr_1_4705_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915715%26sr%3D8-4705-spons%26psc%3D1&qualifier=1596915715&id=5369091505972001&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg99_1?ie=UTF8&adId=A05556532YMG0ZIDRQ93S&url=%2FVeirdo-Sleeve-Round-

https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg100_1?ie=UTF8&adId=A01632561H2EXW9H30M6A&url=%2FBULLMER-Halfsleeve-Printed-Cotton-Tshirt%2Fdp%2FB08CPSK6HJ%2Fref%3Dsr_1_4762_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915720%26sr%3D8-4762-spons%26psc%3D1&qualifier=1596915720&id=3802432503457249&widgetName=sp_btf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg100_1?ie=UTF8&adId=A02876552IHU0KNRLSWRY&url=%2FLee-X-Line-Printed-T-Shirt-L41368CB0XLN_Jsw-Navy%2Fdp%2FB07WG5V1JV%2Fref%3Dsr_1_4763_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915720%26sr%3D8-4763-spons%26psc%3D1&qualifier=1596915720&id=3802432503457249&widgetName=sp_btf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg100_1?ie=UTF8&adId=A00684273SZPO31QF6IVU&url=%2FLee-X-Line-Printed-T-Shirt-L41345CB0XLN_Jsw-Akr%2Fdp%2FB07WKDR86K%2Fref%3Dsr_1_4764_sspa%3Fdchild%3D1%26keywords%3Dmen%2Btshirt%26qid%3D1596915720%26s

In [45]:
import pandas as pd
data_tshirt = pd.read_csv("men-tshirt.csv")

In [46]:
filtered_tshirt = data_tshirt[['SN','Name','Rating','Reviews','Price']]

In [47]:
import numpy as np
filtered_tshirt = filtered_tshirt.replace('-',np.nan)
filtered_tshirt = filtered_tshirt.dropna()
filtered_tshirt['Rating'] = filtered_tshirt['Rating'].str.replace("\sout of 5 stars", "")

In [48]:
filtered_tshirt

,SN,Name,Rating,Reviews,Price
0,1,Adidas Men's Solid Regular fit T-Shirt,3.1,2,"1,199"
2,3,Adidas Men's Solid Regular fit T-Shirt,1.0,1,"1,499"
3,4,Adidas Men's Regular fit T-Shirt,3.4,3,"1,079"
4,5,Scott International Men's Regular Fit T-Shirt ...,3.5,831,399
5,6,Allen Solly Men's Polo,4.1,9458,699
...,...,...,...,...,...
1594,1595,Elements by Peter England Men's Slim fit Polo,4.0,1,479
1595,1596,Lee X-Line Men's Slim fit T-Shirt,3.4,9,383
1596,1597,AD & AV Men's Regular Fit T-Shirt,3.0,6,401
1602,1603,Lee X-Line Men's Slim fit T-Shirt,3.7,3,383


In [53]:
filtered_tshirt['Reviews'] = filtered_tshirt['Reviews'].astype(int)

In [56]:
filtered_tshirt['Rating'] = filtered_tshirt['Rating'].astype(float)

In [59]:
filtered_tshirt['Score'] = (filtered_tshirt['Rating']/5)*(filtered_tshirt['Reviews']/filtered_tshirt['Reviews'].sum())

In [61]:
filtered_tshirt

,SN,Name,Rating,Reviews,Price,Score
0,1,Adidas Men's Solid Regular fit T-Shirt,3.1,2,"1,199",0.000012
2,3,Adidas Men's Solid Regular fit T-Shirt,1.0,1,"1,499",0.000002
3,4,Adidas Men's Regular fit T-Shirt,3.4,3,"1,079",0.000020
4,5,Scott International Men's Regular Fit T-Shirt ...,3.5,831,399,0.005595
5,6,Allen Solly Men's Polo,4.1,9458,699,0.074593
...,...,...,...,...,...,...
1594,1595,Elements by Peter England Men's Slim fit Polo,4.0,1,479,0.000008
1595,1596,Lee X-Line Men's Slim fit T-Shirt,3.4,9,383,0.000059
1596,1597,AD & AV Men's Regular Fit T-Shirt,3.0,6,401,0.000035
1602,1603,Lee X-Line Men's Slim fit T-Shirt,3.7,3,383,0.000021


In [65]:
filtered_tshirt.sort_values('Score',ascending=False).to_csv("sorted_tshirt_100_amazon")

In [68]:
filtered_tshirt.drop_duplicates(keep=False,inplace=True) 